In [ ]:
import pandas
from openeye import oechem, oespruce

# Loading protein, splitting, and aligning to reference

In [ ]:
pdb_fn = "input_pdbs/5WKK.pdb"
## Load molecule to extract
ifs = oechem.oemolistream()
ifs.SetFlavor(
    oechem.OEFormat_PDB,
    oechem.OEIFlavor_PDB_Default
    | oechem.OEIFlavor_PDB_DATA
    | oechem.OEIFlavor_PDB_ALTLOC,
)
ifs.open(pdb_fn)
complex_mol = oechem.OEGraphMol()
oechem.OEReadMolecule(ifs, complex_mol)
ifs.close()

## Keep only highest occupancy alt location
alf = oechem.OEAltLocationFactory(complex_mol)
if alf.GetGroupCount() != 0:
    print(complex_mol.NumAtoms())
    alf.MakePrimaryAltMol(complex_mol)
    print(complex_mol.NumAtoms())

In [ ]:
## Test splitting
lig_mol = oechem.OEGraphMol()
prot_mol = oechem.OEGraphMol()
water_mol = oechem.OEGraphMol()
oth_mol = oechem.OEGraphMol()

## Make splitting split out covalent ligands
## TODO: look into different covalent-related options here
opts = oechem.OESplitMolComplexOptions()
opts.SetSplitCovalent(True)
opts.SetSplitCovalentCofactors(True)
print(
    oechem.OESplitMolComplex(lig_mol, prot_mol, water_mol, oth_mol, complex_mol)
)

print(
    complex_mol.NumAtoms(),
    lig_mol.NumAtoms(),
    prot_mol.NumAtoms(),
    water_mol.NumAtoms(),
    oth_mol.NumAtoms(),
)

In [ ]:
## Load ref PDB molecule
ref_fn = "input_pdbs/reference.pdb"
ref_complex_mol = oechem.OEGraphMol()
ifs.open(ref_fn)
oechem.OEReadMolecule(ifs, ref_complex_mol)
ifs.close()
print(ref_complex_mol.NumAtoms())

## Perform splitting for ref
ref_lig_mol = oechem.OEGraphMol()
ref_prot_mol = oechem.OEGraphMol()
ref_water_mol = oechem.OEGraphMol()
ref_oth_mol = oechem.OEGraphMol()
print(
    oechem.OESplitMolComplex(
        ref_lig_mol, ref_prot_mol, ref_water_mol, ref_oth_mol, ref_complex_mol
    )
)
print(
    ref_complex_mol.NumAtoms(),
    ref_lig_mol.NumAtoms(),
    ref_prot_mol.NumAtoms(),
    ref_water_mol.NumAtoms(),
    ref_oth_mol.NumAtoms(),
)

In [ ]:
## Align new protein to ref
aln_res = oespruce.OESuperposeResults()
superpos = oespruce.OESuperpose()
superpos.SetupRef(ref_prot_mol)
superpos.Superpose(aln_res, prot_mol)
print(f"RMSD: {aln_res.GetRMSD()}")

prot_mol_aligned = prot_mol.CreateCopy()
print(aln_res.Transform(prot_mol_aligned))

## Save new aligned molecule for inspection
out_fn = "test/5WKK_split_aligned.pdb"
ofs = oechem.oemolostream()
ofs.SetFlavor(oechem.OEFormat_PDB, oechem.OEOFlavor_PDB_Default)
ofs.open(out_fn)
oechem.OEWriteMolecule(ofs, prot_mol_aligned)
ofs.close()

# Load in new ligand and create DesignUnit

In [ ]:
## Load ligand molecule
sdf_fn = f"fragalysis/aligned/Mpro-P2007_0A/Mpro-P2007_0A.sdf"
ifs = oechem.oemolistream()
ifs.SetFlavor(
    oechem.OEFormat_SDF,
    oechem.OEIFlavor_SDF_Default,
)
ifs.open(sdf_fn)
coords_mol = oechem.OEGraphMol()
oechem.OEReadMolecule(ifs, coords_mol)
ifs.close()
print(coords_mol.NumAtoms())

## Load CDD compound tracker
compound_df = pandas.read_csv(
    "fragalysis/extra_files/Mpro_compound_tracker_csv.csv"
)

## Get SMILES from compound tracker
idx = compound_df["Dataset"] == "Mpro-P2007"
smiles = compound_df.loc[idx, "SMILES"].values[0]
new_lig_mol = oechem.OEGraphMol()
oechem.OESmilesToMol(new_lig_mol, smiles)
print(new_lig_mol.NumAtoms())

In [ ]:
## Make copy of new_lig_mol so we don't mess up the atom ordering in the
##  actual molecule
temp_mol = new_lig_mol.CreateCopy()

## Reorder the atoms and bonds so the iterate together
oechem.OECanonicalOrderAtoms(coords_mol)
oechem.OECanonicalOrderBonds(coords_mol)
oechem.OECanonicalOrderAtoms(temp_mol)
oechem.OECanonicalOrderBonds(temp_mol)

## Copy coordinates from the SDF-loaded mol into the SMILES-generated one
## Build new coordinate dict with the proper numbering for the
##  smiles-based molecule
new_lig_mol_coords = {}
coords_mol_coords = coords_mol.GetCoords()
for a1, a2 in zip(temp_mol.GetAtoms(), coords_mol.GetAtoms()):
    print(a1, a2)
    idx1 = a1.GetIdx()
    idx2 = a2.GetIdx()
    new_lig_mol_coords[idx1] = coords_mol_coords[idx2]
new_lig_mol.SetCoords(new_lig_mol_coords)

# Brief aside to see if transferring coordinates works
(run to here)

In [ ]:
from openeye import oedepict

oedepict.OEPrepareDepiction(new_lig_mol_test)
oedepict.OEPrepareDepiction(smiles_mol_test)
width, height = 600, 600

opts = oedepict.OE2DMolDisplayOptions(width, height, oedepict.OEScale_AutoScale)
opts.SetAtomPropertyFunctor(oedepict.OEDisplayAtomIdx())
opts.SetAtomPropLabelFont(oedepict.OEFont(oechem.OEDarkGreen))

disp = oedepict.OE2DMolDisplay(new_lig_mol_test, opts)
oedepict.OERenderMolecule("test1.png", disp)

disp = oedepict.OE2DMolDisplay(smiles_mol_test, opts)
oedepict.OERenderMolecule("test2.png", disp)

In [ ]:
## Build new coordinate dict with the proper numbering for the
##  smiles-based molecule
smiles_mol_coords = {}
new_lig_mol_coords = new_lig_mol_test.GetCoords()
for a1, a2 in zip(new_lig_mol_test.GetAtoms(), smiles_mol_test.GetAtoms()):
    idx1 = a1.GetIdx()
    idx2 = a2.GetIdx()
    smiles_mol_coords[idx2] = new_lig_mol_coords[idx1]
smiles_mol_test.SetCoords(smiles_mol_coords)

## Add Hs to both to be able to see double bonds in PDB
print(oechem.OEAddExplicitHydrogens(new_lig_mol_test))
print(oechem.OEAddExplicitHydrogens(smiles_mol_test))
## Update H coordinates to be 3D (not sure why this is necessary,
##  I think only for smiles_mol_test)
print(oechem.OESet3DHydrogenGeom(new_lig_mol_test))
print(oechem.OESet3DHydrogenGeom(smiles_mol_test))

## Save both as PDB to load and see if it worked
ofs = oechem.oemolostream()
ofs.SetFlavor(oechem.OEFormat_PDB, oechem.OEOFlavor_PDB_Default)

ofs.open("test/original_ligand.pdb")
oechem.OEWriteMolecule(ofs, new_lig_mol_test)
ofs.close()

ofs.open("test/coord_copied_ligand.pdb")
oechem.OEWriteMolecule(ofs, smiles_mol_test)
ofs.close()
## It works!

# End aside
(run from here)

In [ ]:
## Add explicit Hs to protein and new ligand to prep
prot_mol_prep = prot_mol_aligned.CreateCopy()
new_lig_prep = new_lig_mol.CreateCopy()
print(oechem.OEAddExplicitHydrogens(prot_mol_prep))
print(oechem.OEAddExplicitHydrogens(new_lig_prep))
## Update H coordinates to be 3D (not sure why this is necessary)
print(oechem.OESet3DHydrogenGeom(new_lig_prep))

## Re-reorder the atoms and bonds
# oechem.OECanonicalOrderAtoms(new_lig_prep)
# oechem.OECanonicalOrderBonds(new_lig_prep)

In [ ]:
from openeye import oedepict


def PrepareDepiction(mol, clearcoords=False, suppressH=True):
    oechem.OESetDimensionFromCoords(mol)
    oechem.OEPerceiveChiral(mol)

    if mol.GetDimension() != 2 or clearcoords:
        if mol.GetDimension() == 3:
            oechem.OE3DToBondStereo(mol)
            oechem.OE3DToAtomStereo(mol)
        if suppressH:
            oechem.OESuppressHydrogens(mol)
        # oedepict.OEAddDepictionHydrogens(mol)

        oedepict.OEDepictCoordinates(mol)
        oechem.OEMDLPerceiveBondStereo(mol)

    mol.SetDimension(2)
    return True


depict_mol = new_lig_prep.CreateCopy()
PrepareDepiction(depict_mol, False, False)
width, height = 600, 600

opts = oedepict.OE2DMolDisplayOptions(width, height, oedepict.OEScale_AutoScale)
opts.SetAtomPropertyFunctor(oedepict.OEDisplayAtomIdx())
opts.SetAtomPropLabelFont(oedepict.OEFont(oechem.OEDarkGreen))

disp = oedepict.OE2DMolDisplay(depict_mol, opts)
oedepict.OERenderMolecule("new_lig.png", disp)

for a in new_lig_prep.GetAtoms():
    print(a)

In [ ]:
## Make the design unit and save
du = oechem.OEDesignUnit()
print(oespruce.OEMakeDesignUnit(du, prot_mol_prep, new_lig_prep))
print(du.HasProtein(), du.HasLigand())
du_out_fn = "test/5WKK_new_lig.oedu"
if du.HasProtein() and du.HasLigand():
    oechem.OEWriteDesignUnit(du_out_fn, du)

In [ ]:
## Save protein + new ligand as PDB
new_complex_mol = oechem.OEGraphMol()
print(du.GetProtein(new_complex_mol))
print(new_complex_mol.NumAtoms())
print(du.GetLigand(new_complex_mol))
print(new_complex_mol.NumAtoms())
print(
    du.GetComponents(
        new_complex_mol,
        oechem.OEDesignUnitComponents_Protein
        | oechem.OEDesignUnitComponents_Ligand,
    )
)
print(new_complex_mol.NumAtoms())
ofs.open("test/5WKK_new_lig.pdb")
oechem.OEWriteMolecule(ofs, new_complex_mol)
ofs.close()

## Also save just lig to check CONECT records
ofs.open("test/5WKK_new_lig_only.pdb")
oechem.OEWriteMolecule(ofs, new_lig_prep)
ofs.close()

# Re-docking of new ligand

In [ ]:
from kinoml.docking.OEDocking import pose_molecules

# score_pose = True
docked_molecules = pose_molecules(du, [new_lig_prep.CreateCopy()])
print(docked_molecules)

In [ ]:
mol = new_lig_prep.CreateCopy()
# print(oechem.OEClearChiralPerception(mol))
# print(oechem.OEPerceiveChiral(mol, True))
print(oechem.OE3DToAtomStereo(mol))

In [ ]:
print(mol.HasPerceived(oechem.OEPerceived_Chiral))
print(oechem.OEClearChiralPerception(mol))
print(mol.HasPerceived(oechem.OEPerceived_Chiral))
print(oechem.OEPerceiveChiral(mol))
print(mol.HasPerceived(oechem.OEPerceived_Chiral))
for a in mol.GetAtoms():
    if a.IsChiral():
        print(a)

In [ ]:
print(new_lig_mol_coords)
for a in new_lig_prep.GetAtoms():
    #     print(a, [str(nbr) for nbr in a.GetAtoms()])
    print(a, a.GetIdx(), new_lig_prep.GetCoords()[a.GetIdx()])

print("-----")

for a in coords_mol.GetAtoms():
    print(a, a.GetIdx(), coords_mol.GetCoords()[a.GetIdx()])